In [5]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import re
import logging

# Helper Functions

In [37]:

def to_dates_list(dates_dictionary):
    dates_list = []
    for date, date_range in dates_dictionary.items():
        dates_list.append(date)
        i = 1 
        sign = lambda a: (a>0) - (a<0)
        while(abs(date_range)>0):
            new_date = pd.to_datetime(date) + (sign(date_range) * timedelta(days=i))
            dates_list.append(new_date.strftime("%Y-%m-%d"))
            i = i+1
            date_range = date_range-(1*(sign (date_range)))
           
                    
    return list(set(dates_list))


date_list = to_dates_list({'2020-03-27':-1,'2020-02-14':1})
date_list


['2020-03-26', '2020-03-27', '2020-02-14', '2020-02-15']

# Data Parsing Function 

This is a pilot for creating a function, which recieves as an input a date and intially returns the climate values
It also has an example of finding the average cases based on range of dates.


In [39]:
file_path = 'sample_data.xlsx'
file_path2 = 'CSSE_confirmed_with_GSOD_wx.csv'
date_range_dict = {'2020-03-27':1,'2020-02-14':1}
#date_range_dict = None


def DataParser(file_path, date_range_dict, incubation_duration=5, region = None, attr_extract=None,_attr_date = 'date_ymd', _attr_country = 'Country/Region'):
    
    # create logger with 'spam_application'
    logger = logging.getLogger('sample')
    logger.setLevel(logging.INFO)
    logger.propagate = False
    # create file handler
    fh = logging.FileHandler('dataparsing.log',mode='w')
    fh.setLevel(logging.INFO)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)    
    logger.addHandler(fh)
    logger.info('Info about the most recent run')
    
   
    
    if attr_extract == None:
        return None
               
    #FileOpen - according to file extension
    if file_path.endswith(".xlsx"):
        xlsx = pd.ExcelFile(file_path)
        df = pd.read_excel(xlsx)
    elif file_path.endswith(".csv"):
        df = pd.read_csv(file_path, parse_dates=['date_ymd'])
    
    
     #RegionSelection - if region parameter is not None, choose the region
    df_extract = df
    if region != None:
        mask_prev = (df_extract[_attr_country] == None)
        for reg in region:
            mask = mask_prev | (df_extract[_attr_country] == reg) 
            mask_prev = mask    
        df_extract = df.loc[mask]
    else:
        df_extract = df
    
    #DateParsing - create a mask to choose the desired dates (According to list and incubation period)
    df_final = pd.DataFrame() #creates a new dataframe that's empty
    if date_range_dict != None:
        date_list = to_dates_list(date_range_dict)
        for date in date_list:
            #get the incubation period of the date
            curr_date_df = df_extract.loc[:,_attr_date: _attr_country]
            end_date = (pd.to_datetime(date)-timedelta(days=incubation_duration)) 
            mask = (df_extract[_attr_date] >= end_date) & (df_extract[_attr_date] <= date)
            incubation_df_extract = df_extract.loc[mask] #extracts the incubation period of the date

            att_df_final = incubation_df_extract.groupby([_attr_country],as_index=False).agg(attr_extract)
            att_df_final.insert(0,_attr_date,date,True)
            df_final = df_final.append(att_df_final, ignore_index = True) # ignoring index is optional

    return df_final

df = DataParser(file_path2,date_range_dict,region=['United Arab Emirates','Antigua and Barbuda'], attr_extract={'confirmed_cases':['mean','min'],'mean_temp': ['max']})
df

date_ymd Country/Region confirmed_cases       mean_temp
                                          mean  min        max
0    2020-03-27    Afghanistan       73.666667   40  58.633333
1    2020-03-27        Albania      137.000000   89  56.300000
2    2020-03-27        Algeria      295.500000  201  60.977358
3    2020-03-27        Andorra      181.500000  113        NaN
4    2020-03-27         Angola        3.166667    2  78.000000
..          ...            ...             ...  ...        ...
683  2020-02-15     Uzbekistan        0.000000    0  41.152381
684  2020-02-15      Venezuela        0.000000    0  81.372727
685  2020-02-15        Vietnam       16.000000   16        NaN
686  2020-02-15         Zambia        0.000000    0        NaN
687  2020-02-15       Zimbabwe        0.000000    0  76.000000

[688 rows x 5 columns]